<a href="https://colab.research.google.com/github/aaronjoel/DeepUnderstandingOfDeepLearning/blob/main/DUDL_MeasurePerformance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import libraries
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# New!
import time

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline as mb
mb.set_matplotlib_formats('svg')

## Import and process the data

In [3]:
# import dataset (comes with colab!)
data = np.loadtxt(open('sample_data/mnist_train_small.csv', 'rb'), delimiter=',')

# extract labels (number IDs) and remove from data
data

array([[6., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.],
       [7., 0., 0., ..., 0., 0., 0.],
       ...,
       [2., 0., 0., ..., 0., 0., 0.],
       [9., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.]])

In [4]:
labels = data[:, 0]
data = data[:, 1:]

# normalize the data to a range of [0 1]
dataNorm = (data - data.min()) / (data.max() - data.min())

dataNorm

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
dataNorm2 = data / data.max()
dataNorm2

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
dataNorm.all() == dataNorm2.all()

np.True_

## Create train/test groups using DataLoader

In [8]:
# Step 1: convert to tensor
dataT = torch.tensor( dataNorm ).float()
labelsT = torch.tensor( labels ).long()

In [10]:
# Step 2: use scikit-learn to split the data
train_data, test_data, train_labels, test_labels = train_test_split(dataT, labelsT, test_size=0.15, random_state=42)

In [11]:
train_data.shape, test_data.shape

(torch.Size([17000, 784]), torch.Size([3000, 784]))

In [12]:
# Step 3: convert into PyTorch datasets
train_data = torch.utils.data.TensorDataset(train_data, train_labels)
test_data  = torch.utils.data.TensorDataset(test_data, test_labels)

In [13]:
# Step 4: translate into dataloader objects
batch_size = 32
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(dataset=test_data, batch_size=test_data.tensors[0].shape[0])

In [15]:
train_loader.dataset.tensors

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([2, 6, 0,  ..., 9, 2, 7]))

In [16]:
test_loader.dataset.tensors

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([9, 5, 2,  ..., 5, 7, 4]))

In [19]:
test_data.tensors[0].shape[0]

3000

In [21]:
test_loader.dataset.tensors[0].shape[0]

3000

## Create the DL model

In [27]:
# create a class for the model
def createTheMNISTNet():

  class mnistNet(nn.Module):

    def __init__(self):
      super().__init__()

      # input layer
      self.input = nn.Linear(784, 64)

      # hidden layer
      self.fc1 = nn.Linear(64, 32)
      self.fc2 = nn.Linear(32, 32)

      # output layer
      self.output = nn.Linear(32, 10)

    def forward(self, x):
      x = F.relu( self.input(x) )
      x = F.relu( self.fc1(x) )
      x = F.relu( self.fc2(x) )
      return self.output(x)

  # create the model instance
  net = mnistNet()

  # loss function
  loss_fun = nn.CrossEntropyLoss()

  # optimizer
  optimizer = torch.optim.Adam(params=net.parameters(), lr=0.01)

  return net, loss_fun, optimizer

## Create a function that trains the model

In [32]:
def function2trainTheModel():

  # Start the timer!
  timerInFunction = time.process_time()

  # number of epochs
  num_epochs = 10

  # create a new model
  net, loss_fun, optimizer = createTheMNISTNet()

  # initialise losses
  losses = torch.zeros(num_epochs)
  trainAcc = []
  testAcc = []

  # loop over epochs
  for epoch in range(num_epochs):
    # loop over training data batches
    batchAcc = []
    batchLoss = []
    for X, y in train_loader:
      # forward pass and loss
      yhat = net(X)
      loss = loss_fun(yhat, y)

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # loss from this batch
      batchLoss.append(loss.item())

      # compute accuracy
      matches = torch.argmax(yhat, axis=1) == y    # booleans (false/true)
      matchesNumeric = matches.float()              # convert to numbers (0/1)
      accuracyPct = 100 * torch.mean(matchesNumeric) # average and x100
      batchAcc.append( accuracyPct )             # add to list of accuracies
    # end of batch loop...

    # now that we've trained through the batches, get their average training accuracy
    trainAcc.append( np.mean(batchAcc) )

    # and get average losses across the batches
    losses[epoch] = np.mean(batchLoss)

    # test accuracy
    X, y = next(iter(test_loader))  # extract X, y from test dataloader
    with torch.no_grad(): # deactivates autograd
      yhat = net(X)

    # Compare the following really long line of code to the training accuracy lines
    testAcc.append( 100*torch.mean( (torch.argmax(yhat, axis=1) == y).float() ) )

    # Finally, report the epoch number, computation time, and accuracy
    comp_time = time.process_time() - timerInFunction
    print(f"Epoch {epoch+1}/{num_epochs}, elapsed time: {comp_time:.2f} sec, test accuracy: {testAcc[-1]:.0f}%")

  # end epochs

  # function output
  return trainAcc, testAcc, losses, net

## Run the model and show the results

In [36]:
# now run a second timer over repeated iterations

# Start the timer! (note the different variable name)
timerOutsideFunction = time.process_time()

for i in range(10):
  function2trainTheModel()

TotalExperimentTime = time.process_time() - timerOutsideFunction
TotalMinutes = TotalExperimentTime // 60
TotalSeconds = TotalExperimentTime - (TotalMinutes * 60)

print(f"\n\nTotal elapsed experiment time: {TotalMinutes} minutes and {TotalSeconds} seconds.")

Epoch 1/10, elapsed time: 1.69 sec, test accuracy: 91%
Epoch 2/10, elapsed time: 3.25 sec, test accuracy: 93%
Epoch 3/10, elapsed time: 4.83 sec, test accuracy: 92%
Epoch 4/10, elapsed time: 6.34 sec, test accuracy: 95%
Epoch 5/10, elapsed time: 8.35 sec, test accuracy: 94%
Epoch 6/10, elapsed time: 10.19 sec, test accuracy: 94%
Epoch 7/10, elapsed time: 11.77 sec, test accuracy: 94%
Epoch 8/10, elapsed time: 13.33 sec, test accuracy: 95%
Epoch 9/10, elapsed time: 14.89 sec, test accuracy: 95%
Epoch 10/10, elapsed time: 16.57 sec, test accuracy: 95%
Epoch 1/10, elapsed time: 1.34 sec, test accuracy: 92%
Epoch 2/10, elapsed time: 3.03 sec, test accuracy: 94%
Epoch 3/10, elapsed time: 5.14 sec, test accuracy: 94%
Epoch 4/10, elapsed time: 6.74 sec, test accuracy: 93%
Epoch 5/10, elapsed time: 8.57 sec, test accuracy: 94%
Epoch 6/10, elapsed time: 10.19 sec, test accuracy: 93%
Epoch 7/10, elapsed time: 12.28 sec, test accuracy: 94%
Epoch 8/10, elapsed time: 13.82 sec, test accuracy: 95%
E

In [38]:
def function2trainTheModel2():

  # Start the timer!
  timerInFunction = time.process_time()

  # number of epochs
  num_epochs = 10

  # create a new model
  net, loss_fun, optimizer = createTheMNISTNet()

  # initialise losses
  losses = torch.zeros(num_epochs)
  trainAcc = []
  testAcc = []

  # loop over epochs
  for epoch in range(num_epochs):
    # loop over training data batches
    batchAcc = []
    batchLoss = []
    for X, y in train_loader:
      # forward pass and loss
      yhat = net(X)
      loss = loss_fun(yhat, y)

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # loss from this batch
      batchLoss.append(loss.item())

      # compute accuracy
      matches = torch.argmax(yhat, axis=1) == y    # booleans (false/true)
      matchesNumeric = matches.float()              # convert to numbers (0/1)
      accuracyPct = 100 * torch.mean(matchesNumeric) # average and x100
      batchAcc.append( accuracyPct )             # add to list of accuracies
    # end of batch loop...

    # now that we've trained through the batches, get their average training accuracy
    trainAcc.append( np.mean(batchAcc) )

    # and get average losses across the batches
    losses[epoch] = np.mean(batchLoss)

    # test accuracy
    X, y = next(iter(test_loader))  # extract X, y from test dataloader
    with torch.no_grad(): # deactivates autograd
      yhat = net(X)

    # Compare the following really long line of code to the training accuracy lines
    testAcc.append( 100*torch.mean( (torch.argmax(yhat, axis=1) == y).float() ) )

    # Finally, report the epoch number, computation time, and accuracy
    comp_time = time.process_time() - timerInFunction
    if epoch == 4:
      print(f"Epoch {epoch+1}/{num_epochs}, elapsed time: {comp_time:.2f} sec, test accuracy: {testAcc[-1]:.0f}%")

  # end epochs

  # function output
  return trainAcc, testAcc, losses, net

In [39]:
# now run a second timer over repeated iterations

# Start the timer! (note the different variable name)
timerOutsideFunction = time.process_time()

for i in range(10):
  function2trainTheModel2()

TotalExperimentTime = time.process_time() - timerOutsideFunction
TotalMinutes = TotalExperimentTime // 60
TotalSeconds = TotalExperimentTime - (TotalMinutes * 60)

print(f"\n\nTotal elapsed experiment time: {TotalMinutes:.0f} minutes and {TotalSeconds:.0f} seconds.")

Epoch 5/10, elapsed time: 8.15 sec, test accuracy: 93%
Epoch 5/10, elapsed time: 8.25 sec, test accuracy: 95%
Epoch 5/10, elapsed time: 7.75 sec, test accuracy: 94%
Epoch 5/10, elapsed time: 8.17 sec, test accuracy: 94%
Epoch 5/10, elapsed time: 8.30 sec, test accuracy: 94%
Epoch 5/10, elapsed time: 7.86 sec, test accuracy: 94%
Epoch 5/10, elapsed time: 8.49 sec, test accuracy: 95%
Epoch 5/10, elapsed time: 7.96 sec, test accuracy: 94%
Epoch 5/10, elapsed time: 7.92 sec, test accuracy: 94%
Epoch 5/10, elapsed time: 8.75 sec, test accuracy: 94%


Total elapsed experiment time: 2.0 minutes and 46 seconds.
